<div align='center'>
МІНІСТЕРСТВО ОСВІТИ І НАУКИ УКРАЇНИ

Національний аерокосмічний університет
"Харківський авіаційний інститут"

Факультет радіотехніки, комп’ютерних систем і інфокомунікацій

Кафедра інформаційно-комунікаційних технологій ім. О. О. Зеленського


ПРАКТИЧНА РОБОТА №4

з курсу "Захист інформації в інфокомунікаціях"

на тему "Пошук мережевих вразливостей кінцевого пристрою"

<div align='right'>
Виконав: студент 538 курсу

групи № 538

спеціальності 126 "Інформаційні системи і технології" (залишити необхідне)

Лопатін Д. С.

Прийняв: доцент каф 504

Єремеєв О.І.

Національна шкала: __________

Кількість балів: _____

Оцінка: ECTS _____ 
</div>

Харків - 2025
</div>

## Практична робота №4. Пошук мережевих вразливостей кінцевого пристрою

**Мета роботи**: ознайомитися з основами роботи мережних протоколів, навчитися використовувати бібліотеку Scapy мови програмування Python для формування запитів для діагностики і сканування пристроїв мережі.

**Вихідні дані**: бібліотека Scapy, програмне забезпечення nmap, nslookup і traceroute(tracert).

**Результат**: програмний код, що дозволяє: 1) виявляти активні пристрої мережі, 2) проводити їх сканування на наявність працюючих мережних служб, 3) робити DNS-запити, 4) визначати маршрут передачі пакетів до заданого вузла.

### Теоретичні основи

Мережі - перша ланка захисту від спрямованих зовнішніх атак та розповсюдження шкідливого забезпечення (віруси, хробаки тощо), які в свою чергу можуть бути використані для виклику різноманітних мережних атак зсередини. \
Забезпечення захисту мережі на належному рівні потребує як розуміння принципів роботи мережі (і протоколів), так і аудит мережі з метою виявлення вразливостей її та пристроїв для подальшого виправлення. \
Пошук працюючих пристроїв мережі, їх сканування для визначення переліку активних служб, а також виявлення їх вразливостей – основне завдання захисту мережі (як і мета атака розвідки). Для цього використовують декілька основних мережних протоколів: ICMP, ARP, TCP, UDP переважно до обчислювальної техніки, а також протоколи маршрутизації та управління мережею - до мережних пристроїв (виходить за рамки даного курсу). 

Поточна практична робота спрямована на ознайомлення з роботою мережі, її основних протоколів і передбачає вирішення наступних задач:
1) пошук активних пристроїв мережі;
2) сканування виявлених пристроїв на  задіяні служби;
3) отримання за допомогою `Nmap` додаткової інформації про служби на активних пристроях, яка може вказувати на наявні вразливості;
4) перевірка роботи DNS-служби і пошук по доменному імені;
5) визначення маршрутів передачі даних до визначеного вузла і всі проміжні маршрутизатори.

Поточна робота є продовженням практичної роботи №3, в якій розглядалися питання щодо використання пакету Scapy, формування пакетів, налаштування доступу до мережі, ІСМР-запити тощо. Тому всі питання і проблеми щодо вказаних тематик були розглянуті в попередній роботі і в практичній роботі №4 вони не розкриваються.


#### Необхідне і рекомендоване програмне забезпечення

1. **Системні утиліти** `ping`, `traceroute`(`tracert` в Windows), `nslookup` є складовими будь-якої операційної системи і не потребують встановлення.

2. **Jupyter-notebook** - рекомендоване ПЗ для організації програмного коду і його взаємодії з командним рядком операційної системи (налаштування наведені в практичній роботі №2).

3. **Бібліотека Scapy** для реалізації мережних пакетів і створення програмного коду Python для взаємодії з мережею (налаштування наведені в практичній роботі №3).
 
4. **Аналізатор пакетів Wireshark** – необов’язкове ПЗ, що за необхідності може бути використане для налагодження доступу до мережі (використання розглянуто в практичній роботі №3).

5. **Nmap** - програмне забезпечення для дослідження та аудиту безпеки мереж та виявлення активних мережних сервісів. Nmap використовує як протокол ICMP, так і TCP, UDP та інші. 

**Встановлення**:
- для Windows - завантажити зі сторінки https://nmap.org/download.html 
- для Linux - в терміналі виконати команди
    - для дистрибутивів на основі Debian/Ubuntu: `sudo apt install nmap`
    - для дистрибутивів на основі RedHat: `sudo dnf install nmap`
    - або слідувати інструкції зі сторінки https://nmap.org/download.html 


### Практична частина

#### Зміст

- Частина 1. Вивчення ПЗ Nmap і сканування мережі
- Частина 2. Використання протоколів ARP, TCP і ІСМР для сканування мережі і пристроїв
- Частина 3. Перевірка роботи служби DNS
- Частина 4. Визначення маршруту до вузла

#### Частина 1. Вивчення ПЗ Nmap і сканування мережі

**Завдання**: ознайомитися з ПЗ Nmap, провести сканування мережі на наявність активних служб та отримання їх інформації

##### Крок 1. Встановлення Nmap
Встановити `Nmap` згідно рекомендацій в теоретичних основах

In [1]:
from requests.packages import target
from scapy.layers.inet import ICMP, TCP
from scapy.layers.l2 import ARP
!sudo pacman -S nmap

resolving dependencies...
looking for conflicting packages...

Packages (1) nmap-7.97-1

Total Installed Size:  25,12 MiB
Net Upgrade Size:       0,00 MiB

:: Proceed with installation? [Y/n] ^C

Interrupt signal received



##### Крок 2. Ознайомлення з можливостями Nmap
Запустити з командного рядка та вивчити можливості програми. `Help` та інструкцію з використання можна вивести за командою:

In [5]:
!nmap -h

Nmap 7.97 ( https://nmap.org )
Usage: nmap [Scan Type(s)] [Options] {target specification}
TARGET SPECIFICATION:
  Can pass hostnames, IP addresses, networks, etc.
  Ex: scanme.nmap.org, microsoft.com/24, 192.168.0.1; 10.0.0-255.1-254
  -iL <inputfilename>: Input from list of hosts/networks
  -iR <num hosts>: Choose random targets
  --exclude <host1[,host2][,host3],...>: Exclude hosts/networks
  --excludefile <exclude_file>: Exclude list from file
HOST DISCOVERY:
  -sL: List Scan - simply list targets to scan
  -sn: Ping Scan - disable port scan
  -Pn: Treat all hosts as online -- skip host discovery
  -PS/PA/PU/PY[portlist]: TCP SYN, TCP ACK, UDP or SCTP discovery to given ports
  -PE/PP/PM: ICMP echo, timestamp, and netmask request discovery probes
  -PO[protocol list]: IP Protocol Ping
  -n/-R: Never do DNS resolution/Always resolve [default: sometimes]
  --dns-servers <serv1[,serv2],...>: Specify custom DNS servers
  --system-dns: Use OS's DNS resolver
  --traceroute: Trace hop pat

##### Крок 3. Визначення мережних налаштувань пристрою
Виконати команди командного рядка(терміналу) для визначення налаштувань мережі: 
- для Windows - `ipconfig /all`
- для Linux - `ip -br address`
- або зайти в графічний інтерфейс налаштувань мережі

In [2]:
!ip -br address

lo               UNKNOWN        127.0.0.1/8 ::1/128 
enp3s0           UP             192.168.18.58/24 fe80::f063:57c2:420b:39d/64 
docker0          DOWN           172.17.0.1/16 
br-61dd16293aff  DOWN           172.18.0.1/16 


##### Крок 4. Сканування своєї мережі
Виконати сканування всієї локальної мережі відповідно до прикладів з **кроку 2** і враховуючи ІР-адресу і маску мережі поточного пристрою. Також слід використовувати опції, що дозволять провести сканування і вивести необхідну інформацію.

Приклади опцій з кроку №2:
- Ключ `-v` ... `-vvvvv` вказує на різний рівень відображення інформації про сканування
- Ключ `-A` вказує на запуск скриптів з визначення версій ОС та ПЗ ("Enable OS detection, version detection, script scanning, and traceroute")

**Примітка**: Результати роботи `Nmap` (виявлені ІР-адреси і номери портів) є еталонними значеннями, які мають бути досягнені в частині 2 практичної роботи.

In [5]:
!nmap -v -A 192.168.18.58/24

Starting Nmap 7.97 ( https://nmap.org ) at 2025-11-13 16:26 +0100
NSE: Loaded 158 scripts for scanning.
NSE: Script Pre-scanning.
Initiating NSE at 16:26
Completed NSE at 16:26, 0.00s elapsed
Initiating NSE at 16:26
Completed NSE at 16:26, 0.00s elapsed
Initiating NSE at 16:26
Completed NSE at 16:26, 0.00s elapsed
Initiating ARP Ping Scan at 16:26
Scanning 255 hosts [1 port/host]
Completed ARP Ping Scan at 16:26, 2.10s elapsed (255 total hosts)
Initiating Parallel DNS resolution of 255 hosts. at 16:26
Completed Parallel DNS resolution of 255 hosts. at 16:27, 39.98s elapsed
Nmap scan report for 192.168.18.0 [host down]
Nmap scan report for 192.168.18.2 [host down]
Nmap scan report for 192.168.18.3 [host down]
Nmap scan report for 192.168.18.4 [host down]
Nmap scan report for 192.168.18.5 [host down]
Nmap scan report for 192.168.18.6 [host down]
Nmap scan report for 192.168.18.7 [host down]
Nmap scan report for 192.168.18.8 [host down]
Nmap scan report for 192.168.18.9 [host down]
Nmap s

##### Крок 5. Збереження результатів роботи по частині 1
Зберегти для звіту результати виконання кроків №2-4 - налаштування мережі і результати виконання команд програмою Nmap. \
*Не потрібно у разі використання даного шаблону jupyter-notebook*

#### Частина 2. Використання протоколів ARP, TCP і ICMP для сканування мережі і пристроїв
**Завдання**: провести сканування власної мережі за допомогою Python i Scapy і виявити працюючі вузли і TCP-служби в них.

##### Крок 1. Виконання ICMP або ARP запитів на всі вузли мережі
Провести сканування мережі на наявність активних пристроїв шляхом надсилання пакетів (ICMP або ARP) на всі доступні ІР-адреси мережі враховуючи налаштування ІР-адреси і маски мережі поточного пристрою.

**Рекомендації**:
- перелік адрес можна задати як через цикл, так і за допомогою діапазонного визначення ІР-адреси;
- налаштуйте обмеження на очікування відповіді на запит, щоб уникнути нескінченного очікування на відповідь від неіснуючих вузлів мережі (1-2 секунди або менше);
- ICMP-запит потребує попереднього ARP-запиту для визначення МАС-адреси і формування заголовку Ethernet. Для зменшення затримок на визначення МАС-адреси неіснуючих вузлів можна відправляти ICMP-запит з широкомовною МАС-адресою отримувача `ff:ff:ff:ff:ff:ff` або реалізувати формування ARP-пакету.

In [16]:
from scapy.all import *

ans, un = srp(Ether(dst="ff:ff:ff:ff:ff:ff")/ARP(pdst="192.168.18.0/24"), timeout=1)
ans.show()

Begin emission
**
Finished sending 256 packets
.
Received 3 packets, got 2 answers, remaining 254 packets
0000 Ether / ARP who has 192.168.18.1 says 192.168.18.58 ==> Ether / ARP is at e0:4b:a6:3c:e7:6c says 192.168.18.1 / Padding
0001 Ether / ARP who has 192.168.18.12 says 192.168.18.58 ==> Ether / ARP is at 4e:52:56:08:78:1b says 192.168.18.12 / Padding


##### Крок 2. Відображення результатів сканування ІР-адрес
Вивести перелік виключно активних вузлів мережі.

In [19]:
active_hosts = [rcv[ARP].psrc for _, rcv in ans]

print("Active hosts:")
for ip in active_hosts:
    print(ip)


Active hosts:
192.168.18.1
192.168.18.12


##### Крок 3. TCP-scanning
Для активних вузлів зробити TCP-scanning використовуючи SYN-ключ (варто ознайомитися в матеріалах лекції та на офіційній сторінці Scapy). 

Щонайменше мають бути перевірені порти 20-150 і 443, а бажано весь діапазон до 1000 або більше (максимальне допустиме значення номера порту – 65535). Якщо використовуються сервери баз даних, то варто і їх включити у перелік портів, що скануються.

In [21]:
ports = list(range(1000))
ans, unans = sr(IP(dst=active_hosts)/TCP(dport=ports, flags="S"), timeout=1, retry=0, inter=0.01, verbose=0)

##### Крок 4. Відображення результатів сканування портів активних пристроїв
Вивести перелік активних ТСР-портів, використовуючи наступну підказку з документації:
```python
ans.nsummary( lfilter=lambda s,r: (r.haslayer(TCP) and (r.getlayer(TCP).flags & 2)) )
```
**Примітка**: На активному вузлі по протоколу ТСР відповідь буде надходити на кожен запит, але дійсно існуюча служба має відповісти на прапорець `SYN` в заголовку ТСР встановленим прапорцем `АСК` у відповіді, що і перевіряє наведений приклад лямбда-фільтру для методу `nsummary()`.


In [29]:
ans.nsummary(lfilter=lambda s, r: r.haslayer(TCP) and (r[TCP].flags & 2))

print("\nOpen ports:")

open_map = {}

for s, r in ans:
    if r.haslayer(TCP) and (r[TCP].flags & 0x12) == 0x12:
        open_map.setdefault(r[IP].src, []).append(r[TCP].sport)

for host, ports in open_map.items():
    print(f"{host}: {sorted(set(ports))}")

0050 IP / TCP 192.168.18.58:ftp_data > 192.168.18.1:domain S ==> IP / TCP 192.168.18.1:domain > 192.168.18.58:ftp_data SA / Padding
0077 IP / TCP 192.168.18.58:ftp_data > 192.168.18.1:www_http S ==> IP / TCP 192.168.18.1:www_http > 192.168.18.58:ftp_data SA / Padding

Open ports:
192.168.18.1: [53, 80]


##### Крок 5. Збереження результатів роботи по частині 2
Зберегти для звіту програмний код (кроки №1 і 3) і результати їх виконання (кроки №2 і 4). \
*Не потрібно у разі використання даного шаблону jupyter-notebook*

#### Частина 3. Перевірка роботи служби DNS
**Завдання**: перевірити роботу служби DNS локальної мережі.

DNS - служба доменних імен, що забезпечує перетворення доменного імені певного сайту на його ІР-адресу. Для цього використовується одноіменний протокол 7-го рівня, який інкапсулюється в UDP-дейтаграми з портом призначення рівним 53.

Варіанти адрес призначення запитів 

|Варіант|Сайт|Кількість запитів N|
|---|---|---|
|1|google.com|6
|2|yahoo.com|3
|3|facebook.com|4
|4|python.org|5
|5|youtube.com|6
|6|wikipedia.org|3
|7|amazon.com|4
|8|live.com|5
|9|bing.com|6
|10|instagram.com|3
|11|stackoverflow.com|4
|12|whatsapp.com|5
|13|twitter.com|6
|14|telegram.com|3
|15|viber.com|4
|16|ukr.net|5
|17|accuweather.com|6
|18|sinoptik.ua|3
|19|gismeteo.ua|4
|20|i.ua|5
|21|www.netacad.com|6

##### Крок 1. Обрання адреси призначення за варіантом по списку
Відповідно до порядкового номеру в списку групи оберіть сайт для виконання роботи.

##### Крок 2. Очищення кешу DNS
Всі проміжні мережні пристрої (поточний пристрій, маршрутизатор) працюють в якості кешуючого DNS-серверу – на певний проміжок часу зберігають результат запитів щодо певних доменних імен для повторного використання. Тому для коректності наступних кроків необхідно видалити збережені результати: 
- у Windows: `ipconfig /flushdns`
- в Linux (з системою ініціації systemd): `resolvectl flush-caches`

In [34]:
!resolvectl flush-caches

##### Крок 3. Використання nslookup
В усіх операційних системах для перевірки роботи і служби DNS і визначення ІР-адреси використовується службова утиліта `nslookup`:

In [31]:
!nslookup python.org

Server:		192.168.18.1
Address:	192.168.18.1#53

Non-authoritative answer:
Name:	python.org
Address: 151.101.64.223
Name:	python.org
Address: 151.101.128.223
Name:	python.org
Address: 151.101.192.223
Name:	python.org
Address: 151.101.0.223
Name:	python.org
Address: 2a04:4e42:400::223
Name:	python.org
Address: 2a04:4e42:200::223
Name:	python.org
Address: 2a04:4e42:600::223
Name:	python.org
Address: 2a04:4e42::223



Результат буде мати приблизно наступний формат:
```shell
Server:  XiaoQiang
Address:  192.168.31.1

Non-authoritative answer:
Name:    google.com
Addresses:  2a00:1450:401b:804::200e
          216.58.209.14
```

##### Крок 4. Очищення кешу DNS
Виконайте повторно **крок 2** для очищення кешу DNS

##### Крок 5. Реалізація запиту через Scapy
Створіть і надішліть засобами Scapy пакет на маршрутизатор використовуючи протоколи UDP і DNS для компонування пакету використовуючи наступний приклад зі своїми параметрами:
```python
ans = sr1(IP(dst="192.168.31.1")/UDP(sport=RandShort(), dport=53)/
DNS(rd=1,qd=DNSQR(qname="google.com",qtype="A")))
```
**Примітка**: 
- rd=1 – прапорець заголовку DNS, що вказує на потребу в рекурсивних запитах (DNS-сервер, якщо не знає відповіді, має перенаправити на наступний (вищий по ієрархії) DNS-сервер);
- qname="google.com" – доменне ім’я, для якого створено запит;
- qtype="A" – тип запиту для отримання ІР-адреси по вказаному доменному імені.

In [36]:
ans = sr1(IP(dst="192.168.18.1")/UDP(sport=RandShort(), dport=53)/DNS(rd=1,qd=DNSQR(qname="python.org",qtype="A")))

Begin emission

Finished sending 1 packets
*
Received 1 packets, got 1 answers, remaining 0 packets


##### Крок 6. Відображення результату DNS-запиту
1. Виведіть з заголовків пакету значення доменного імені, що задавалося, і йому відповідну ІР-адресу. 
2. Порівняйте результат з командою `nslookup` і зробіть висновки.

In [41]:
qname = ans[DNS].qd.qname.decode()
ips = [ ans[DNS].an[i].rdata for i in range(ans[DNS].ancount) if ans[DNS].an[i].type == 1 ]
print(f"{qname} -> {ips}")

python.org. -> ['151.101.0.223', '151.101.64.223', '151.101.128.223', '151.101.192.223']


##### Крок 7. Збереження результатів роботи по частині 3
Зберегти для звіту результати роботи кроків №3,5 і 6. \
*Не потрібно у разі виконанні завдання у даному шаблоні jupyter-notebook*

#### Частина 4. Визначення маршруту до вузла 
**Завдання**: для вузла, обраного в **частині 3** виконати запит на визначення маршруту передачі пакетів за допомогою стандартних утиліт `tracert` (Windows) / `traceroute` (Linux).

##### Крок 1. Використання traceroute
Зробити запит до раніше обраного веб-ресурсу, використовуючи системну утиліту `tracert` (Windows) або `traceroute` (Linux) для визначення маршруту передачі пакетів (перелік маршрутизаторів, які забезпечували маршрутизацію пакетів).

In [46]:
!traceroute python.org

traceroute to python.org (151.101.64.223), 30 hops max, 60 byte packets
 1  _gateway (192.168.18.1)  0.464 ms  0.407 ms  0.594 ms
 2  10.170.224.1 (10.170.224.1)  4.297 ms  4.391 ms  4.475 ms
 3  10.10.50.10 (10.10.50.10)  12.670 ms  8.311 ms  13.337 ms
 4  242.red.88.37.46.procono.es (46.37.88.242)  18.187 ms  18.606 ms  13.661 ms
 5  * * *
 6  * * *
 7  * * *
 8  * * *
 9  * * *
10  * * *
11  * * *
12  * * *
13  * * *
14  * * *
15  * * *
16  * * *
17  * * *
18  * * *
19  * * *
20  * * *
21  * * *
22  * * *
23  * * *
24  * * *
25  * * *
26  * * *
27  * * *
28  * * *
29  * * *
30  * * *


##### Крок 2. Реалізація трасування за допомогою Scapy
Трасування забезпечується покроковим збільшенням поля `ttl` (дозволена кількість переходів). Кожний маршрутизатор при обробці пакета зменшує значення даного поля на 1. Якщо значення зменшується до 0, то той маршрутизатор видаляє пакет і повертає службове ІСМР-повідомлення про відкидання даного пакета. Відповідно, покрокове збільшення значення поля з 1 і вище дозволяє пройти кожного разу на один маршрутизатор далі, отримуючи інформацію від наступного.

Виконайте завдання використовуючи формат для виведення результату з попереднього кроку:
1. Сніфінг з попередньої практики для замірювання часу
2. Інкріментне збільшення поля `ttl` 
3. Ускладнене завдання: DNS-запити для визначення доменного імені маршрутизатора кожного переходу. Бажано ознайомитися з PTR-записом і зворотнім перетворенням ІР-адреси у доменне ім'я.

In [48]:
def ptr(ip):
    ans = sr1(IP(dst="8.8.8.8")/UDP(sport=RandShort(), dport=53)/
              DNS(rd=1, qd=DNSQR(qname=".".join(ip.split('.')[::-1])+".in-addr.arpa", qtype="PTR")),
              timeout=1, verbose=0)
    if ans and ans.haslayer(DNS) and ans[DNS].ancount:
        n = ans[DNS].an.rdata
        return n.decode() if isinstance(n, bytes) else n


target = "python.org"
max_hops = 20

print(f"Tracing route to {target}:\n")

for ttl in range(1, max_hops+1):
    t0 = time.time()
    r = sr1(IP(dst=target, ttl=ttl)/ICMP(), timeout=1, verbose=0)
    dt = round((time.time()-t0)*1000, 2)
    if not r:
        print(f"{ttl:2}  *  timeout")
        continue
    ip = r.src
    name = ptr(ip)
    print(f"{ttl:2}  {ip:<16} {dt} ms" + (f"  {name}" if name else ""))
    if r.type == 0:
        print("\nTrace complete.")
        break

Tracing route to python.org:

 1  192.168.18.1     38.1 ms
 2  10.170.224.1     56.42 ms
 3  10.10.50.10      59.86 ms
 4  46.37.88.242     73.16 ms  242.red.88.37.46.procono.es.
 5  23.235.34.108    72.85 ms
 6  151.101.0.223    63.12 ms

Trace complete.


##### Крок 3. Збереження результатів роботи по частині 4
Зберегти для звіту результати роботи кроків №1-2. \
*Не потрібно у разі виконанні завдання у даному шаблоні jupyter-notebook*

##### Крок 4. Оформлення звіту
Оформити звіт про виконання роботи і навести в ньому результати по кожні частині даної практичної роботи. У разі використання шаблону jupyter notebook достатньо надати сам файл блокноту зі збереженими результатами виконання блоків коду.

##### Оцінювання роботи
Максимальна оцінка в 100 балів, її складові:
- частина 1		– до 15 балів
- частина 2	 	– до 30 балів
- частина 3 		– до 10 балів
- частина 4 		– до 30 балів (35 балів за ускладнене)
- належним чином підготовлений звіт з титульною сторінкою (або шаблон jupyter notebook) з реалізацією і виконанням всіх завдань – до 15 балів

**Примітка**: за відсутності звіту (шаблону) робота не приймається
